In [1]:
import sys,os
import data_processing as dp
import ecc_tools as tools
import timeit
# import pydca-ER module
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder

from pydca.erdca import erdca
from pydca.sequence_backmapper import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.msa_trimmer.msa_trimmer import MSATrimmerException
from pydca.dca_utilities import dca_utilities
import numpy as np
import pickle

# Import Bio data processing features 
import Bio.PDB, warnings
from Bio.PDB import *
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning

warnings.filterwarnings("error")
warnings.simplefilter('ignore', BiopythonWarning)
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', ResourceWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

data_path = '/home/eclay/Pfam-A.full'
preprocess_path = '/home/eclay/DCA_ER/biowulf/pfam_ecc/'

In [2]:
# Define Simulation Parameters
pfam_id = 'PF03456' # best socre by ER
# AUC should be ~.981436
pfam_id = 'PF00186' # best socre by ER


cpus_per_job = 8

print("Calculating DI for %s using %d (of %d) threads "%(pfam_id,cpus_per_job-4,cpus_per_job))

                                               


Calculating DI for PF00186 using 4 (of 8) threads 


In [3]:
# Read in Reference Protein Structure
pdb = np.load('%s/%s/pdb_refs.npy'%(data_path,pfam_id))                                                                                                                   
# convert bytes to str (python 2 to python 3)                                                                       
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0])      for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])
ipdb = 0
tpdb = int(pdb[ipdb,1])
print('Ref Sequence # should be : ',tpdb-1)

# Load Multiple Sequence Alignment
s = dp.load_msa(data_path,pfam_id)

# Load Polypeptide Sequence from PDB as reference sequence
print(pdb[ipdb,:])
pdb_id = pdb[ipdb,5]                                                                              
pdb_chain = pdb[ipdb,6]                                                                           
pdb_start,pdb_end = int(pdb[ipdb,7]),int(pdb[ipdb,8])                                             
pdb_range = [pdb_start-1, pdb_end]
#print('pdb id, chain, start, end, length:',pdb_id,pdb_chain,pdb_start,pdb_end,pdb_end-pdb_start+1)                        

#print('download pdb file')                                                                       
pdb_file = pdb_list.retrieve_pdb_file(str(pdb_id),file_format='pdb')                              
#pdb_file = pdb_list.retrieve_pdb_file(pdb_id)     

chain = pdb_parser.get_structure(str(pdb_id),pdb_file)[0][pdb_chain] 
ppb = PPBuilder().build_peptides(chain)                                                       
#    print(pp.get_sequence())
print('peptide build of chain produced %d elements\n\n'%(len(ppb)))                               

matching_seq_dict = {}
poly_seq = list()
for i,pp in enumerate(ppb):
    for char in str(pp.get_sequence()):
        poly_seq.append(char)                                     
print('PDB Polypeptide Sequence: \n',poly_seq)

poly_seq_range = poly_seq[pdb_range[0]:pdb_range[1]]
print('PDB Polypeptide Sequence (In Proteins PDB range len=%d): \n'%len(poly_seq_range),poly_seq_range)

pp_msa_file_range, pp_ref_file_range = tools.write_FASTA(poly_seq_range, s, pfam_id, number_form=False,processed=False,path='./pfam_ecc/',nickname='range')

pp_msa_file, pp_ref_file = tools.write_FASTA(poly_seq, s, pfam_id, number_form=False,processed=False,path='./pfam_ecc/')


Ref Sequence # should be :  68
shape of s (import from msa.npy):
 (7750, 918)
shape of s (after UTF-8 decode):
 (7750, 918)
['PF00186' '69' 'Q5KZ26_GEOKA' '1' '160' '1ZDR' 'B' '1' '160']
Structure exists: '/home/eclay/DCA_ER/zd/pdb1zdr.ent' 
peptide build of chain produced 1 elements


PDB Polypeptide Sequence: 
 ['M', 'I', 'S', 'H', 'I', 'V', 'A', 'M', 'D', 'E', 'N', 'R', 'V', 'I', 'G', 'K', 'D', 'N', 'R', 'L', 'P', 'W', 'H', 'L', 'P', 'A', 'D', 'L', 'A', 'Y', 'F', 'K', 'R', 'V', 'T', 'M', 'G', 'H', 'A', 'I', 'V', 'M', 'G', 'R', 'K', 'T', 'F', 'E', 'A', 'I', 'G', 'R', 'P', 'L', 'P', 'G', 'R', 'D', 'N', 'V', 'V', 'V', 'T', 'G', 'N', 'R', 'S', 'F', 'R', 'P', 'E', 'G', 'C', 'L', 'V', 'L', 'H', 'S', 'L', 'E', 'E', 'V', 'K', 'Q', 'W', 'I', 'A', 'S', 'R', 'A', 'D', 'E', 'V', 'F', 'I', 'I', 'G', 'G', 'A', 'E', 'L', 'F', 'R', 'A', 'T', 'M', 'P', 'I', 'V', 'D', 'R', 'L', 'Y', 'V', 'T', 'K', 'I', 'F', 'A', 'S', 'F', 'P', 'G', 'D', 'T', 'F', 'Y', 'P', 'P', 'I', 'S', 'D', 'D', 'E', 'W', 'E', 'I',

In [ ]:
trimmed_data_outfile = preprocess_path+'MSA_%s_Trimmed.fa'%pfam_id
print('Pre-Processing MSA')
pfam_dict = {}


try:
    print('\n\nPre-Processing MSA with Range PP Seq\n\n')
    trimmer = msa_trimmer.MSATrimmer(
        pp_msa_file_range, biomolecule='PROTEIN', 
        refseq_file=pp_ref_file_range
    )
    pfam_dict['ref_file'] = pp_ref_file_range
except:
    print('\nDidnt work, using full PP seq\nPre-Processing MSA wth PP Seq\n\n')

    
# Adding the data_processing() curation from tools to erdca.
try:
    trimmed_data = trimmer.get_msa_trimmed_by_refseq(remove_all_gaps=True)
    print('\n\nTrimmed Data: \n',trimmed_data[:10])


except(MSATrimmerException):
    ERR = 'PPseq-MSA'
    print('Error with MSA trimms\n%s\n'%ERR)
    sys.exit()
    
    
#write trimmed msa to file in FASTA format
with open(trimmed_data_outfile, 'w') as fh:
    for seqid, seq in trimmed_data:
        fh.write('>{}\n{}\n'.format(seqid, seq))
fh.close()

s_index = list(np.arange(len(''.join(seq))))


In [ ]:
print('s_index : ',s_index,'\n')

print('Initializing ER instance\n\n')
# Compute DI scores using Expectation Reflection algorithm
erdca_inst = erdca.ERDCA(
    #preprocessed_data_outfile,
    trimmed_data_outfile,
    'PROTEIN',
    refseq_file=pp_ref_file_range,
    s_index = s_index,
    pseudocount = 0.5,
    num_threads = cpus_per_job-4,
    seqid = 0.8)

In [ ]:
print('Running ER simulation\n\n')
# Compute average product corrected Frobenius norm of the couplings
start_time = timeit.default_timer()
#erdca_DI = erdca_inst.compute_sorted_DI(LAD=False,init_w = couplings)
erdca_DI = erdca_inst.compute_sorted_DI(LAD=True,init_w =False) # initializes with DCA couplings
run_time = timeit.default_timer() - start_time
print('ER run time:',run_time)

for site_pair, score in erdca_DI[:10]:
    print(site_pair, score)

with open('erdca_DI_%s.pickle'%(pfam_id), 'wb') as f:
    pickle.dump(erdca_DI, f)
f.close()

In [ ]:
# Save processed data dictionary and FASTA file
print('s shape (msa): ',s.shape)
print('s_index shape: ',len(s_index))
pfam_dict['msa'] = s  
pfam_dict['s_index'] = s_index
pfam_dict['processed_msa'] = trimmed_data 
pfam_dict['s_ipdb'] = tpdb
pfam_dict['cols_removed'] = []

input_data_file = preprocess_path+"%s_DP_laderdca.pickle"%(pfam_id)
